# Comparison between DLC and Sleap


Sleap provides a couple of small videos at the [sleap repo under the test data folder](https://github.com/talmolab/sleap/tree/develop/tests/data/videos), as well as a sample video from the [sleap datasets repo](https://github.com/talmolab/sleap-datasets) that is more refined and also contains some labels. DeepLabCut provides example videos in their [git repo](https://github.com/DeepLabCut/DeepLabCut/tree/main/examples), and we will be using the [example mice video](https://github.com/DeepLabCut/DeepLabCut/blob/main/examples/Reaching-Mackenzie-2018-08-30/videos/reachingvideo1.avi) for this comparison. 

Comparison pipeline:

- Generate 20 frames to label, train on 80% (16 frames) and test on 20% (4 frames)
- Train model
- Predict on entire video

Evaluation metrics:

- Pixel error on test labeled frames
- Visualize Predictions Labeled video


### How to follow this notebook

To follow through this notebook, you must have a sleap conda environment installed as described in the README. Then change the kernel of this notebook to `Python [sleap]`. Now you will be able to run the commands in the notebook within the sleap conda environment. 

### Table of contents

- [Sleap Model training](#SLEAP-TRAINING)
    - [Train-Test Split Preprocessing](#Train-Test-Split-Preprocessing)
    - [Train-Test Split in Terminal](#Train-Test-in-Terminal)
- [(Optional) Prediction-assisted labeling in Sleap GUI](#(Optional)-Prediction-assisted-labeling-in-GUI)
- [Sleap Model evaluation in notebook](#SLEAP-EVALUATION)
- [Sleap Predictions post-processing](#SLEAP-PREDICTION-post-processing)
    - [Sleap Generate Prediction video](#SLEAP-Generate-Predictions-Video)
    
    
[Use DLC labels in Sleap (in progress)](#Use-DLC-labels-in-Sleap-(in-progress))

# SLEAP TRAINING

Use the GUI for labeling and prediction-assisted training as usual. It is easy to do prediction-assisted training and inference in the GUI, although it is certainly doable through the terminal as well. The only complication with the GUI comes  when you would like to do a train-test split, and there are several approaches to do it manually, either through the terminal, the GUI, or the jupyter notebook. 

Train-test split is tricky in Sleap becuase by default all labeled frames will be used for training. Suppose we generate 20 suggested frames, then we want to use 16 of them for training and 4 of them for testing. The default validation fraction is 0.1, which means that 0.1 of the training frames (i.e. 2 frames in this case) will not be used for training directly, instead will be used for tuning hyperparameters and early stopping. Model metrics would be generated based on its performance on the 4 testing frames. 

[Train config file documentation](https://sleap.ai/develop/api/sleap.nn.config.data.html#sleap.nn.config.data.LabelsConfig)

- Training labels: used for training
- Validation labels: not used for training, used for hyperparameters and early stopping, default sample 0.1 from training labels
- Test labels: not used for training, used for benchmarking

## Train-Test Split Preprocessing

Since Sleap does not provide a native train-test split in the GUI, we need to generate separate train and test label files manually. 

First, create a folder inside the Home Folder to store all SLEAP related files for this project, for example `sleap-mouse-topview`. 

Then in the SLEAP GUI, generate 20 frames and save as `labels.20frames.slp` file, where 16 will be training frames and 4 will be testing frames. 

- Open a new project in the GUI using the `labels.20frames.slp` file, label 4 frames, and `Export Label Package` to a `labels.20frames4labeled.pkg.slp` file in Home Folder.
- Open a new project in the GUI using the `labels.20frames.slp` file, label the remaining 16 frames, and `Export Label Package` to a `labels.20frames16labeled.slp` file in Home Folder.
- In the same project, click `Predict - Run Training` in the toolbox, and mofidify train configurations: e.g. configure 10 epoch for each model, predict on Nothing, set run prefix to "appstream_train16_test4". 
- DO NOT click run training. Instead, click `Export Training Job` and save the zip file to the Home Folder with a descriptive name. This will be the training job folder name, for example `labels.appstream0inference20frames16labeled.slp.training_job`.

Go to Appstream's home folder and unzip the training job folder. It should contain a `labels.20frames16labeled.pkg.slp` file and some training job config files. Move the `labels.20frames4labeled.pkg.slp` file into this folder as well.


NOTE: Save labels via `Predict - Export Labels` and not `File - Save As` if you want to use them elsewhere (e.g. on local computer). If directly Save As .. in GUI after labeling, the labeled frames in the `.slp` file will reference the original video file with its appstream location, which wouldn't be available if you moved the files. However, if saved via Export Labels, the labeled frames in the `.pkg.slp` file will only reference the `.pkg.slp` file itself.

In [ ]:
# !cd models/appstream_noinference_train16_test4230721_210307.centroid

### Train-Test in Terminal

Similar file processing but in the `appstream-terminal` training job folder. Change directory to training job folder, then in terminal, run 

    sleap-train centroid.json labels.20frames16labeled.pkg.slp --test labels.20frames4labeled.pkg.slp
    sleap-train centered_instance.json labels.20frames16labeled.pkg.slp --test labels.20frames4labeled.pkg.slp

Models will be generated under the training job folder in `sleap-mouse-topview/appstream-terminal/models`. 

Run inference on the entire video using the trained models. 

    sleap-track C:\s3-mount\dlc-sample-videos\mouse-movement-videos\mouse-top-view.mp4 \
        -m models\appstream_noinference_train16_test4_fromscript230721_210307.centroid \
        -m models\appstream_noinference_train16_test4_fromscript230721_210307.centered_instance \
        -o sleap-mouse-topview-video.predictions.slp

### Train in GUI

Similar file processing but in the `appstream-modifyjson` training job folder. Change directory to training job folder, and modify both json files so that the training label and test labels paths point to the `.pkg.slp` files saved in the folder. 

```
"training_labels": "D:/PhotonUser/My Files/Home Folder/sleap-mouse-topview/appstream_modifyjson/labels.20frames16labeled.pkg.slp",
"validation_labels": null,
"validation_fraction": 0.1,
"test_labels": "D:/PhotonUser/My Files/Home Folder/sleap-mouse-topview/appstream_modifyjson/labels.20frames4labeled.pkg.slp",
```

Use these json files in the GUI train configurations window. 

Models will be generated under the sleap folder in `sleap-mouse-topview/models`. 

### Train in Notebook

In the jupyter notebook, change directory to the training job folder and start training. 

In [69]:
# Uncomment following lines and run once to change notebook working directpory
# %cd "My Files\Home Folder\sleap-mouse-topview"
# %cd "labels.appstream0inference20frames16labeled.slp.training_job"

In [ ]:
!sleap-train centroid.json labels.20frames16labeled.pkg.slp --test labels.20frames4labeled.pkg.slp
!sleap-train centered_instance.json labels.20frames16labeled.pkg.slp --test labels.20frames4labeled.pkg.slp

Trained model results will be generated under a `models` folder in the training job folder. 

## (Optional) Prediction-assisted labeling in GUI

Open `predictions.slp` file in GUI, correct predicted frames that are incorrect. Save labels to `predictions-relabel.slp` file in training job folder.  

Then, you could run the training entirely in the GUI (method 1), or extract a .slp file with the user-labeled corrections and choose to re-train in the GUI (method 2) or re-train in the terminal as shown above (method 3). You could choose to merge the user-labeled corrections to the original project file with `Files-Merge Data From ...`, in order to train with both the originally labeled frames and the corrected frames. Instructions here reference the [official documentation](https://sleap.ai/develop/guides/merging.html). 

Method 1: In GUI, click `Predict-Run Training`, models will be generated under training job folder in `sleap-mouse-topview/appstream-terminal/models`. 

Method 2: In Sleap GUI toolbar, click `Delete All Predictions...` to delete all predicted instances and be left with user-labeled corrections only. Save the corrected labels to another `.slp` file so that you can merge into another project in the GUI. 

Method 3: You may also use the following python code to generate the `only_relabels.slp` file with only user-labeled corrections, then run training in the terminal as demonstrated above. Note that the file name cannot contain hyphens (`-`). 

    labels = sleap.load_file('preditions-relabel.slp')
    only_relabels = labels.with_user_labels_only()
    only_relabels.save("only_relabels.slp")

## SLEAP EVALUATION

You can analyze the performance of the trained models using the metrics available in the model files. You can follow the steps below in a jupyter notebook or in a similar python environment. If you would like to export resulting model files for analysis somewhere else, run `tar -acf out.zip /path/to/models`, and download the out.zip file to local computer. 

Here is an example view of what the models folder should contain. If you want to analyze the centroid model, go to `models\appstream_noinference_train16_test4230721_210307.centroid`. The test pixel error is available in the `metrics.test.npz` file. 

In [271]:
from pathlib import Path

DIR = Path("C:/Users/louise.xu/Downloads/appstream-terminal")
MODELS_DIR = DIR / "models"
CENTROID_MODEL = MODELS_DIR / "appstream_noinference_train16_test4_fromscript230721_210307.centered_instance"

!dir $CENTROID_MODEL

 Volume in drive C is Windows
 Volume Serial Number is 2E94-A0A3

 Directory of C:\Users\louise.xu\Downloads\appstream-terminal\models\appstream_noinference_train16_test4_fromscript230721_210307.centered_instance

07/25/2023  11:33 AM    <DIR>          .
07/25/2023  11:33 AM    <DIR>          ..
07/25/2023  11:33 AM        69,145,416 best_model.h5
07/25/2023  11:33 AM             5,975 initial_config.json
07/25/2023  11:33 AM            18,040 labels_gt.test.slp
07/25/2023  11:33 AM            20,088 labels_gt.train.slp
07/25/2023  11:33 AM            18,040 labels_gt.val.slp
07/25/2023  11:33 AM            15,944 labels_pr.test.slp
07/25/2023  11:33 AM            16,448 labels_pr.train.slp
07/25/2023  11:33 AM            15,944 labels_pr.val.slp
07/25/2023  11:33 AM             2,147 metrics.test.npz
07/25/2023  11:33 AM             2,471 metrics.train.npz
07/25/2023  11:33 AM             1,991 metrics.val.npz
07/25/2023  11:33 AM             8,200 training_config.json
07/25/2023  11:

Here is how you can load the model metrics for different splits, i.e. train, validation, and test. 

In [272]:
import sleap

#help(sleap.load_metrics)
metrics_val = sleap.load_metrics(CENTROID_MODEL, split="val")
metrics_train = sleap.load_metrics(CENTROID_MODEL, split="train")
metrics_test = sleap.load_metrics(CENTROID_MODEL, split="test")

print("====train====")
print("mAP:", metrics_train["oks_voc.mAP"])
print("pixel error:", metrics_train["dist.avg"])
print("====validation====")
print("mAP:", metrics_val["oks_voc.mAP"])
print("pixel error:", metrics_val["dist.avg"])
print("====test====")
print("mAP:", metrics_test["oks_voc.mAP"])
print("pixel error:", metrics_test["dist.avg"])

====train====
mAP: 0.8126787678767876
pixel error: 1.2829211239840028
====validation====
mAP: 0.02524752475247525
pixel error: 3.9137164844054566
====test====
mAP: 0.06386138613861386
pixel error: 32.214755664959654


## SLEAP PREDICTION post-processing

You can also directly process the predictions made on the entire video using the predictions.slp file. Here is a quick view of what data it contains, but we will convert it into a .h5 file for easier analysis. 

In [243]:
DOWNLOADS = Path("C:/Users/louise.xu/Downloads")
#labels = sleap.load_file(str(TRAIN_JOB / "sleap-mouse-topview-video.predictions-labeled.pkg.slp"))
predictions = Labels.load_file(str(DOWNLOADS / "sleap-mouse-topview-video.predictions.slp"), video_search = str(TRAIN_JOB / "mouse-top-view.mp4"))
print(predictions.predicted_instances[0])
print(predictions.predicted_instances[0].points_and_scores_array)
print(predictions.predicted_instances[0].points)

PredictedInstance(video=Video(filename=C:\Users\louise.xu\Downloads\mouse-top-view.mp4, shape=(2330, 480, 640, 1), backend=MediaVideo), frame_idx=0, points=[head: (77.0, 88.6, 1.02), tailBase: (140.4, 187.7, 0.95)], score=0.94, track=None, tracking_score=0.00)
[[ 76.97714233  88.58641815   1.02416515]
 [140.41918945 187.72146606   0.94858468]]
(PredictedPoint(x=76.97714233398438, y=88.58641815185547, visible=True, complete=False, score=1.024165153503418), PredictedPoint(x=140.419189453125, y=187.72146606445312, visible=True, complete=False, score=0.9485846757888794))


Run the following codeblock or run the command in terminal to convert convert .slp files into .h5 files.

In [227]:
!sleap-convert "C:/Users/louise.xu/Downloads/sleap-mouse-topview-video.predictions.slp" --format analysis --output mouse-labels.h5


Exporting to SLEAP Analysis file...
	track_names: 0
	node_names: 2
	edge_names: 1
	edge_inds: 1
	tracks: (2330, 2, 2, 1)
	track_occupancy: (1, 2330)
	point_scores: (2330, 2, 1)
	instance_scores: (2330, 1)
	tracking_scores: (2330, 1)
	labels_path: C:/Users/louise.xu/Downloads/sleap-mouse-topview-video.predictions.slp
	video_path: C:/Users/louise.xu/Downloads\mouse-top-view.mp4
	video_ind: 0
	provenance: {"model_paths": ["models\\appstream_noinference_train16_test4_fromscript230721_210307.centroid\\training_config.json", "models\\appstream_noinference_train16_test4_fromscript230721_210307.centered_instance\\training_config.json"], "predictor": "TopDownPredictor", "sleap_version": "1.3.0", "platform": "Windows-10-10.0.17763-SP0", "command": "C:\\miniconda3\\envs\\sleap\\Scripts\\sleap-track C:\\s3-mount\\dlc-sample-videos\\mouse-movement-videos\\mouse-top-view.mp4 -m models\\appstream_noinference_train16_test4_fromscript230721_210307.centroid -m models\\appstream_noinference_train16_test

In [230]:
%cd C:\Users\louise.xu\Downloads
import h5py
import matplotlib.pyplot as plt

filename = "mouse-labels.h5" # or your own exported filename

with h5py.File(filename, "r") as f:
    dset_names = list(f.keys())
    locations = f["tracks"][:].T
    node_names = [n.decode() for n in f["node_names"][:]]

print("===filename===")
print(filename)
print()

print("===HDF5 datasets===")
print(dset_names)
print()

print("===locations (tracks) data shape===")
print(locations.shape)
print()

print("===nodes===")
for i, name in enumerate(node_names):
    print(f"{i}: {name}")
print()

C:\Users\louise.xu\Downloads
===filename===
mouse-labels.h5

===HDF5 datasets===
['edge_inds', 'edge_names', 'instance_scores', 'labels_path', 'node_names', 'point_scores', 'provenance', 'track_names', 'track_occupancy', 'tracking_scores', 'tracks', 'video_ind', 'video_path']

===locations (tracks) data shape===
(2330, 2, 2, 1)

===nodes===
0: head
1: tailBase



You can also make a `.csv` file containing the predicted locations from the `.h5` file. This is useful if you would like to import this into DeepLabCut. 

In [256]:
import numpy as np
locations_csv = locations.reshape((2330, 4))
header= ""
for name in node_names:
    header += name + "-x, " + name + "-y, "
np.savetxt("C:/Users/louise.xu/Downloads/sleap-mouse-predictions.csv", locations_csv, delimiter=",", header=header, comments="")

### SLEAP Generate Predictions Video

Once you have the `predictions.slp` file containing predictions of the entire video, you can generate a video file visualizing these prediction instances. 

Method 1 (GUI): Open the `predictions.slp` file. In the GUI toolbar, click `View-Render Video Clip with Instances...`. 

Method 2 (Terminal): Change to the directory containing the `predictions.slp` file. Then enter the following command. I set the fps to be 30, the same as The sample video fps value. Notice that the output name must contain a valid extension such as `.avi`

    sleap-render predictions.slp --fps 30 --output predictions-video.avi

# Use DLC labels in Sleap (in progress)

You can `File-Import-DeepLabCut dataset` to import a DeepLabCut `.csv` file containing the user label coordinates. This will open a Sleap GUI window containing only the labeled video frames. Save this as a `labels.fromdlc.slp` file. 

Next open a new Sleap project and load the raw video file. Then click `File-Merge into Project` to merge the `labels.fromdlc.slp` file into the current project. 

- Run into error during merge : "Labels.skeleton can only be used when there is only a single skeleton saved in the labels. Use Labels.skeletons instead."
